In [1]:
import pandas as pd
import csv
import os
import numpy as np
import nltk
import re

In [2]:
from nltk.tokenize import word_tokenize

In [3]:
path=os.getcwd()
path

'/Users/garima/AISecKG-cybersecurity-dataset/Model/dataprep'

In [4]:
annotatedata = pd.read_csv(path + '/annotated_data_BI.csv')

In [11]:
annotatedata.head()

sentence  \
0                  Attacks can damage public domain.   
1                      IDS means Intrusion Detection   
2              Snort is used for Intrusion Detection   
3                  Snort can detect network attacks.   
4  Snort can also be used as a simple packet logger.   

                                              entity  
0             B-attack can damage B-system I-system.  
1               B-tool means B-technique I-technique  
2         B-tool is used for B-technique I-technique  
3               B-tool can detect B-attack I-attack.  
4  B-tool can also be used as a simple B-tool I-t...

In [17]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [18]:
for ele in annotatedata['sentence']:
    if ele in punc:
        annotatedata['sentence'] = annotatedata['sentence'].replace(ele, "")

In [167]:
regex1 = re.compile('B-')
regex2 = re.compile('I-')

In [19]:
annotatedata.head()

sentence  \
0                  Attacks can damage public domain.   
1                      IDS means Intrusion Detection   
2              Snort is used for Intrusion Detection   
3                  Snort can detect network attacks.   
4  Snort can also be used as a simple packet logger.   

                                              entity  
0             B-attack can damage B-system I-system.  
1               B-tool means B-technique I-technique  
2         B-tool is used for B-technique I-technique  
3               B-tool can detect B-attack I-attack.  
4  B-tool can also be used as a simple B-tool I-t...

In [168]:
df2=pd.DataFrame(columns=['tag'])

In [169]:
for row in annotatedata['entity']:
    for word in row.split():
        res1 = regex1.search(word)
        res2= regex2.search(word)
        if res1==None and res2==None:
            row=row.replace(word,'O')
    #print(row)
    df2 = df2.append(pd.DataFrame([row], columns=['tag']))

In [21]:
df2.to_csv('annotated_BIO.csv')

In [15]:
df = pd.read_csv(path + '/annotated_BIO.csv')

In [16]:
df.head()

DocNo  Sno                                          sentence  \
0    D1    1                  Attacks can damage public domain   
1    D1    2                     IDS means Intrusion Detection   
2    D1    3             Snort is used for Intrusion Detection   
3    D1    4                  Snort can detect network attacks   
4    D1    5  Snort can also be used as a simple packet logger   

                                    tag  
0        B-attack O O B-system I-system  
1      B-tool O B-technique I-technique  
2  B-tool O O O B-technique I-technique  
3          B-tool O O B-attack I-attack  
4    B-tool O O O O O O O B-tool I-tool

In [17]:
# add a blank row after each statement
df_new = pd.DataFrame()
for i, row in df.iterrows():
    df_new = df_new.append(row)
    for _ in range(1):
        df_new = df_new.append(pd.Series(), ignore_index=True)

<ipython-input-17-f767b3ba9cd9>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_new = df_new.append(pd.Series(), ignore_index=True)


In [21]:
df_new['Sno']=df_new['Sno'].fillna(0).astype('int')

In [22]:
df_new.head()

DocNo  Sno                               sentence  \
0    D1    1       Attacks can damage public domain   
1   NaN    0                                    NaN   
2    D1    2          IDS means Intrusion Detection   
3   NaN    0                                    NaN   
4    D1    3  Snort is used for Intrusion Detection   

                                    tag  
0        B-attack O O B-system I-system  
1                                   NaN  
2      B-tool O B-technique I-technique  
3                                   NaN  
4  B-tool O O O B-technique I-technique

In [23]:
#df_new['DocNo']=df_new['DocNo'].fillna(0)

In [26]:
u = df_new.sentence.str.split(expand=True).stack()
v = df_new.tag.str.split(expand=True).stack()

In [27]:
len(u)

8965

In [28]:
len(v)

9010

In [29]:
u

0     0      Attacks
      1          can
      2       damage
      3       public
      4       domain
              ...   
1182  12    received
      13          at
      14         the
      15      server
      16        side
Length: 8965, dtype: object

In [30]:
v

0     0     B-attack
      1            O
      2            O
      3     B-system
      4     I-system
              ...   
1182  12           O
      13           O
      14           O
      15    B-system
      16    I-system
Length: 9010, dtype: object

In [33]:
v[-10:]

1182  7              O
      8     B-function
      9              O
      10        B-tool
      11        B-data
      12             O
      13             O
      14             O
      15      B-system
      16      I-system
dtype: object

In [67]:
df_new['c1'] = df_new['sentence'].str.split().str.len()

In [68]:
df_new['c2'] = df_new['tag'].str.split().str.len()

In [69]:
df_new['c1']=df_new['c1'].fillna(0).astype('int')
df_new['c2']=df_new['c2'].fillna(0).astype('int')

In [70]:
df_new.head()

Unnamed: 0  Unnamed: 0.1 DocNo  Sno                               sentence  \
0           0             0    D1    1       Attacks can damage public domain   
1           1             1   NaN    0                                    NaN   
2           2             2    D1    2          IDS means Intrusion Detection   
3           3             3   NaN    0                                    NaN   
4           4             4    D1    3  Snort is used for Intrusion Detection   

                                    tag  c1  c2  compare  
0        B-attack O O B-system I-system   5   5        1  
1                                   NaN   0   0        1  
2      B-tool O B-technique I-technique   4   4        1  
3                                   NaN   0   0        1  
4  B-tool O O O B-technique I-technique   6   6        1

In [71]:
df_new['compare']= np.where((df_new['c1'] != df_new['c2']), 0,1)

In [72]:
df_new.head()

Unnamed: 0  Unnamed: 0.1 DocNo  Sno                               sentence  \
0           0             0    D1    1       Attacks can damage public domain   
1           1             1   NaN    0                                    NaN   
2           2             2    D1    2          IDS means Intrusion Detection   
3           3             3   NaN    0                                    NaN   
4           4             4    D1    3  Snort is used for Intrusion Detection   

                                    tag  c1  c2  compare  
0        B-attack O O B-system I-system   5   5        1  
1                                   NaN   0   0        1  
2      B-tool O B-technique I-technique   4   4        1  
3                                   NaN   0   0        1  
4  B-tool O O O B-technique I-technique   6   6        1

In [73]:
df_new.to_csv('temp.csv')

In [89]:
df_new = pd.read_csv(path + '/temp.csv')

In [93]:
df_new.head()

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1 DocNo  Sno  \
0           0             0               0    D1    1   
1           1             1               1   NaN    0   
2           2             2               2    D1    2   
3           3             3               3   NaN    0   
4           4             4               4    D1    3   

                                sentence  \
0       Attacks can damage public domain   
1                                  blank   
2          IDS means Intrusion Detection   
3                                  blank   
4  Snort is used for Intrusion Detection   

                                    tag  c1  c2  compare  
0        B-attack O O B-system I-system   5   5        1  
1                                 blank   0   0        1  
2      B-tool O B-technique I-technique   4   4        1  
3                                 blank   0   0        1  
4  B-tool O O O B-technique I-technique   6   6        1

In [75]:
ds.head()

sentence       tag
0  Attacks  B-attack
1      can         O
2   damage         O
3   public  B-system
4   domain  I-system

In [91]:
# insert blanks
df_new['sentence']=df_new['sentence'].fillna('blank')

In [92]:
df_new['tag']=df_new['tag'].fillna('blank')

In [94]:
# split the sentence into words
u = df_new.sentence.str.split(expand=True).stack()
v = df_new.tag.str.split(expand=True).stack()


dsp = pd.DataFrame({
    'sentence': u.values, 
    'tag': v.values, 
})

In [96]:
dsp.to_csv('splitdata.csv')

In [97]:
dsp = pd.read_csv(path + '/splitdata.csv')

In [98]:
dsp.head()

DocNo  Unnamed: 1 sentence       tag
0    D1           0  Attacks  B-attack
1    D1           1      can         O
2    D1           2   damage         O
3    D1           3   public  B-system
4    D1           4   domain  I-system

In [99]:
dsp = dsp.drop('Unnamed: 1', axis=1)

In [102]:
dsp['DocNo'].mask(dsp['sentence'] == 'blank', 'blank', inplace=True)

In [105]:
dsp.to_csv('splitdata.csv')

In [5]:
dsf = pd.read_csv(path + '/splitdata_final.csv')

In [7]:
dsf = dsf.fillna("")
dsf.tag.unique()

array(['B-attack', 'O', 'B-system', 'I-system', '', 'B-tool',
       'B-technique', 'I-technique', 'I-attack', 'I-tool', 'B-data',
       'B-feature', 'I-feature', 'B-function', 'I-function', 'I-data',
       'B-vulnerability', 'I-vulnerability', 'B-app', 'B-securityTeam',
       'I-securityTeam', 'B-attacker', 'I-attacker', 'B-user', 'I-app',
       'I-user'], dtype=object)

In [8]:
dsf= dsf.drop('DocNo', axis=1)

In [9]:
dsf.head()

sentence       tag
0  Attacks  B-attack
1      can         O
2   damage         O
3   public  B-system
4   domain  I-system

In [10]:
# remove the other words
df_entity = dsf[dsf['tag'] != 'O']

In [14]:
len(df_entity)

3833

In [5]:
# schema as per ontology
schema = [
('user', 'uses', 'app'), 
('user', 'uses', 'system'), 
('user', 'uses', 'data'), 
('app', 'uses', 'data'), 
('system', 'uses', 'data'), 
('system', 'has_a', 'tool'), 
('app', 'has_a', 'tool'),
('tool', 'is_part_of', 'system'),
('tool', 'is_part_of', 'app'),
('tool', 'is_part_of', 'tool'),
('tool', 'has_a', 'function'),
('tool', 'has_a', 'feature'),
('tool', 'uses', 'technique'),
('technique', 'has_a', 'tool'),
('function', 'has_a', 'feature'),
('feature', 'is_part_of', 'tool'),
('system', 'has_a', 'feature'),
('app', 'has_a', 'feature'),
('feature', 'uses', 'data'),
('tool', 'uses', 'data'), 
('app', 'can_expose', 'vulnerability'),
('system', 'can_expose', 'vulnerability'),
('data', 'can_expose', 'vulnerability'),
('feature', 'can_expose', 'vulnerability'),
('attacker', 'can_exploit', 'vulnerability'),
('attacker', 'uses', 'feature'),
('attacker', 'uses', 'function'),
('attacker', 'uses', 'tool'),
('attacker', 'implements', 'attack'),
('attacker', 'uses', 'technique'),
('technique', 'implements', 'attack'),
('attacker', 'can_harm', 'app'),
('attacker', 'can_harm', 'data'),
('attacker', 'can_harm', 'system'),
('attack', 'is_part_of', 'attack'), 
('attack', 'can_harm', 'system'),
('attack', 'can_harm', 'app'),
('attack', 'can_harm', 'data'), 
('securityTeam', 'can_analyze', 'app'),
('securityTeam', 'can_analyze', 'data'),
('securityTeam', 'can_analyze', 'system'),
('securityTeam', 'uses', 'tool'),
('securityTeam', 'implements', 'function'),
('securityTeam', 'can_analyze', 'feature'),
('securityTeam', 'uses', 'technique'),
('securityTeam', 'can_detect', 'vulnerability'),
('securityTeam', 'can_analyze', 'attack'),
('tool', 'can_analyze', 'system'),
('tool', 'can_analyze', 'app'),
('tool', 'can_analyze', 'data'),
('tool', 'can_analyze', 'vulnerability'),
('tool', 'can_detect', 'attack'),
('function', 'can_analyze', 'vulnerability'),
('function', 'can_analyze', 'system'),
('function', 'can_analyze', 'app'),
('function', 'can_analyze', 'data'),
('function', 'can_detect', 'attack'),
('feature', 'can_analyze', 'system'),
('feature', 'can_analyze', 'app'),
('feature', 'can_analyze', 'data'),
('feature', 'can_analyze', 'vulnerability') ,
('feature', 'can_detect', 'attack') ,
('technique', 'can_analyze', 'vulnerability'), 
('technique', 'can_detect', 'attack')
]

In [15]:
df_entity.to_csv('temp2.csv')

In [62]:
df_entity.head(10)

sentence          tag
0     Attacks,     B-attack
3      public,     B-system
4      domain,     I-system
5            ,             
6         IDS,       B-tool
8   Intrusion,  B-technique
9   Detection,  I-technique
10           ,             
11      Snort,       B-tool
15  Intrusion,  B-technique

In [31]:
df_entity.sentence

0        Attacks
3         public
4         domain
5               
6            IDS
          ...   
9549        test
9551    firewall
9552        logs
9556      server
9557        side
Name: sentence, Length: 3833, dtype: object

In [61]:
df_entity['sentence']= df_entity['sentence'].apply(lambda x:"{:},".format(x))

<ipython-input-61-262552ce83cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entity['sentence']= df_entity['sentence'].apply(lambda x:"{:},".format(x))


In [63]:
dfc= df_entity.groupby((~df_entity.tag.str.startswith('I')).cumsum(), as_index=False).sum()

In [64]:
dfc.head(10)

sentence                     tag
0              Attacks,                B-attack
1        public,domain,        B-systemI-system
2                     ,                        
3                  IDS,                  B-tool
4  Intrusion,Detection,  B-techniqueI-technique
5                     ,                        
6                Snort,                  B-tool
7  Intrusion,Detection,  B-techniqueI-technique
8                     ,                        
9                Snort,                  B-tool

In [65]:
dfc.to_csv('temp2.csv')

In [6]:
dfc = pd.read_csv(path + '/temp2.csv')

In [73]:
#dfc.dropna(subset=['tag'], inplace=True)

In [78]:
dfc['tag'] = dfc['tag'].str.replace('','')

<ipython-input-78-4e634ccb6785>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dfc['tag'] = dfc['tag'].str.replace('I-?','')


In [92]:
dfc['tag'] = dfc['tag'].str.replace('I-attack',' ')

In [94]:
dfc['tag'] = dfc['tag'].str.replace('I-tool',' ')

In [97]:
dfc['tag'] = dfc['tag'].str.replace('I-system',' ')

In [99]:
dfc['tag'] = dfc['tag'].str.replace('I-feature',' ')
dfc['tag'] = dfc['tag'].str.replace('I-function',' ')
dfc['tag'] = dfc['tag'].str.replace('I-technique',' ')
dfc['tag'] = dfc['tag'].str.replace('I-data',' ')
dfc['tag'] = dfc['tag'].str.replace('I-user',' ')
dfc['tag'] = dfc['tag'].str.replace('I-vulnerability',' ')
dfc['tag'] = dfc['tag'].str.replace('I-app',' ')
dfc['tag'] = dfc['tag'].str.replace('I-securityTeam',' ')
dfc['tag'] = dfc['tag'].str.replace('I-attacker',' ')

In [87]:
import re

In [101]:
dfc['tag'] = dfc['tag'].str.replace('B-',' ')

In [124]:
dfc.to_csv('temp2.csv')

In [115]:
dfc['entity']=list(zip(dfc.sentence, dfc.tag))

In [148]:
# create tuples
entity_list = dfc['entity'].tolist()

In [159]:
#create sublists for triples
from itertools import groupby
key= entity_list.index(' ')

res = [list(sub) for ele, sub in groupby(entity_list, key = bool) if ele]
# temp_idx = entity_list.index(' ')
#res = [entity_list[: temp_idx], entity_list[temp_idx + 1: ]]

In [42]:
dfc= dfc.drop('relation', axis=1)

In [ ]:
dfc['tag']=dfc['tag'].fillna('blank')

In [12]:
dfc['relation']=''

In [13]:
for r in range(len(dfc)-1):
    for i in range(len(schema)):
        if dfc['tag'][r].strip()==schema[i][0] and dfc['tag'][r+1].strip()==schema[i][2]:
            dfc['relation'][r]=(schema[i][1])
            #print(r, schema[i][1])

<ipython-input-13-f5fbdb6c5fad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['relation'][r]=(schema[i][1])


In [16]:
dfc['tag'][0].strip()

'attack'

In [18]:
dfc['tag'][1].strip()

'system'

In [14]:
schema[1][0]

'user'

In [15]:
schema[1][2]

'system'

In [14]:
dfc.head(5)

Unnamed: 0  Unnamed: 0.1              sentence          tag  \
0           0             0              Attacks        attack   
1           1             1        public domain       system    
2           2             2                              blank   
3           3             3                  IDS          tool   
4           4             4  Intrusion Detection    technique    

                                    entity  relation  
0                  ('Attacks ', ' attack')  can_harm  
1           ('public domain ', ' system ')            
2                                                     
3                        ('IDS ', ' tool')      uses  
4  ('Intrusion Detection ', ' technique ')

In [15]:
dfc.to_csv('triple_temp.csv')

In [16]:
triple = []
for i in range(len(dfc)-1):
    if (dfc['relation'][i]=='') is False:
        triple.append((dfc['sentence'][i].strip(),dfc['relation'][i], dfc['sentence'][i+1].strip() ) ) 
    else:
        continue
    #print(triple)

In [17]:
dtriple = pd.DataFrame(triple)

In [18]:
dtriple.to_csv('triples.csv')